In [ ]:
!pip install transformers sentencepiece gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
model_id = "sivakornboonnet/ThaiT5-absa-thai-model"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.3k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def convert_result(text):
    lines = re.findall(r"<SYMBOL>\s*(.*?)\s*<ASPECT>\s*(.*?)\s*<OPINION>\s*(.*?)\s*<(POS|NEG|NEU)>", text)
    if not lines:
        return "<span style='color:red;'>🚫 ไม่พบผลลัพธ์ที่ตรงตามรูปแบบ <SYMBOL> ... <ASPECT> ... <OPINION> ... <POS|NEG|NEU></span>"

    sentiment_color = {
        "POS": ("เชิงบวก", "#4ade80"),   # เขียว
        "NEG": ("เชิงลบ", "#f87171"),   # แดง
        "NEU": ("เชิงกลาง", "#facc15") # เหลือง
    }

    block_style = """
        display: flex;
        flex-wrap: wrap;
        gap: 12px;
        margin-bottom: 16px;
    """

    tag_style = lambda bg: f"""
        background: {bg};
        color: white;
        padding: 8px 14px;
        border-radius: 20px;
        font-weight: bold;
        box-shadow: 0 2px 6px rgba(0,0,0,0.2);
    """

    html_blocks = []
    for symbol, aspect, opinion, sentiment in lines:
        sent_text, sent_bg = sentiment_color[sentiment]

        block = f"""
        <div style="{block_style}">
            <span style="{tag_style('#2563eb')}">หุ้น: {symbol}</span>
            <span style="{tag_style('#0284c7')}">หัวข้อ: {aspect}</span>
            <span style="{tag_style('#9333ea')}">ถ้อยคำ: {opinion}</span>
            <span style="{tag_style(sent_bg)}">แนวโน้ม: {sent_text}</span>
        </div>
        """
        html_blocks.append(block)

    return "".join(html_blocks)

In [ ]:
def analyze_news(text):
    if not text.strip():
        return "<div style='color:red;'>⚠️ กรุณากรอกข้อความข่าวหุ้น</div>"

    try:
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_length=50)
        generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
        cleaned = generated.replace("<pad>", "").replace("</s>", "").strip()
        print("🧠 Output:", cleaned)
        return convert_result(cleaned)
    except Exception as e:
        return f"<div style='color:red;'>❌ Error: {str(e)}</div>"

In [ ]:
custom_css = """
footer {display:none !important}
textarea {font-size: 16px !important;}
button {font-size: 16px !important; font-weight: bold;}
#component-0 {max-width: 800px; margin: auto;}
"""

In [ ]:
iface = gr.Interface(
    fn=analyze_news,
    inputs=gr.Textbox(lines=5, placeholder="พิมพ์หรือวางข่าวหุ้นที่นี่...", label="ข่าวหุ้นภาษาไทย"),
    outputs=gr.HTML(label="ผลลัพธ์การวิเคราะห์"),
    title="🧠 ระบบวิเคราะห์ข่าวหุ้นไทย (ABSA)",
    description="วิเคราะห์แง่มุมของข่าวหุ้นไทยอย่างแม่นยำ พร้อมแสดงผลสรุปที่เข้าใจง่าย",
    submit_btn="วิเคราะห์",
    clear_btn="ล้างข้อมูล",
    allow_flagging="never",
    css=custom_css
)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [ ]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fbe911ce3d463f1ca8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
